# Generate thumbnails from videos

Automatically create preview thumbnails from video files at specific timestamps or intervals.


## Problem

You have video files that need preview thumbnails for galleries, search results, or video players. Manually extracting frames doesn't scale.

| Use case | Videos | Need |
|----------|--------|------|
| Video platform | 10K videos | Preview thumbnails |
| Media library | 5K clips | Gallery previews |
| Content management | 1K uploads | Automated thumbnails |


## Solution

**What's in this recipe:**
- Extract thumbnail at a specific timestamp
- Generate multiple thumbnails per video
- Resize thumbnails to standard dimensions

You add computed columns that extract frames from videos. Thumbnails are generated automatically when you insert new videos.


### Setup


In [ ]:
%pip install -qU pixeltable


In [ ]:
import pixeltable as pxt
from pixeltable.functions import video as video_fn


### Load videos


In [ ]:
# Create a fresh directory
pxt.drop_dir('thumbnail_demo', force=True)
pxt.create_dir('thumbnail_demo')


In [ ]:
# Create table for videos
videos = pxt.create_table('thumbnail_demo.videos', {'video': pxt.Video})


In [ ]:
# Insert sample videos from public S3 bucket
s3_prefix = 's3://multimedia-commons/'
video_paths = [
    'data/videos/mp4/ffe/ffb/ffeffbef41bbc269810b2a1a888de.mp4',
    'data/videos/mp4/ffe/feb/ffefebb41485539f964760e6115fbc44.mp4',
]

videos.insert([{'video': s3_prefix + path} for path in video_paths])


In [ ]:
# View videos
videos.collect()


### Extract thumbnail at timestamp

Extract a single frame at a specific time (e.g., 1 second into the video):


In [ ]:
# Extract frame at 1 second as thumbnail
videos.add_computed_column(
    thumbnail=video_fn.extract_frame(videos.video, timestamp_sec=1.0)
)


In [ ]:
# View thumbnails
videos.select(videos.video, videos.thumbnail).collect()


### Resize thumbnails

Create standard-sized thumbnails for consistent display:


In [ ]:
# Resize thumbnail to 320x180 (16:9 aspect ratio)
videos.add_computed_column(
    thumbnail_small=videos.thumbnail.resize((320, 180))
)


In [ ]:
# View resized thumbnails with dimensions
videos.select(
    videos.thumbnail_small,
    videos.thumbnail_small.width,
    videos.thumbnail_small.height
).collect()


### Multiple thumbnails with FrameIterator

For preview strips or timeline thumbnails, use FrameIterator to extract multiple frames:


In [ ]:
from pixeltable.iterators import FrameIterator

# Create a view with frames extracted at 0.5 fps (one frame every 2 seconds)
frames = pxt.create_view(
    'thumbnail_demo.frames',
    videos,
    iterator=FrameIterator.create(video=videos.video, fps=0.5)
)


In [ ]:
# View extracted frames (multiple per video)
frames.select(frames.frame, frames.pos).head(10)


## Explanation

**Thumbnail extraction methods:**

| Method | Use case |
|--------|----------|
| `extract_frame(timestamp)` | Single thumbnail at specific time |
| `FrameIterator(fps=N)` | Multiple frames at regular intervals |
| `FrameIterator(num_frames=N)` | Fixed number of evenly-spaced frames |

**Common thumbnail sizes:**

| Size | Use case |
|------|----------|
| 120x90 | Small previews, lists |
| 320x180 | Medium thumbnails, cards |
| 640x360 | Large previews, hover states |
| 1280x720 | HD thumbnails, hero images |


## See also

- [Extract frames from videos](./video-extract-frames.ipynb) - Detailed frame extraction guide
- [Load media from S3](./data-import-s3.ipynb) - Import videos from cloud storage
- [Transform images with PIL](./img-pil-transforms.ipynb) - Resize and crop images
